In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from olympus.datasets import Dataset
from olympus.emulators import Emulator

from olympus.models import BayesNeuralNet

In [2]:
res = pickle.load(open('best_scores.pkl', 'rb'))

In [3]:
dataset_names = [
    'oer_plate_4098', 'oer_plate_3851', 'oer_plate_3860', 'oer_plate_3496',
    'p3ht', 'agnp', 
    'thin_film', 'crossed_barrel', 'autoam', 
    'suzuki_i', 'suzuki_ii', 'suzuki_iii', 'suzuki_iv',
]






dataset_params = { 
        'oer_plate_4098': {'out_act': 'sigmoid', 'feature_transform': 'identity', 'target_transform': 'normalize'},
        'oer_plate_3851': {'out_act': 'sigmoid', 'feature_transform': 'identity', 'target_transform': 'normalize'},
        'oer_plate_3860': {'out_act': 'sigmoid', 'feature_transform': 'identity', 'target_transform': 'normalize'},
        'oer_plate_3496': {'out_act': 'sigmoid', 'feature_transform': 'identity', 'target_transform': 'normalize'},
        #
        'p3ht': {'out_act': 'relu', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'thin_film': {'out_act': 'relu', 'feature_transform': 'identity', 'target_transform': 'normalize'},
        'crossed_barrel': {'out_act': 'relu', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'autoam': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'agnp': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        #
        'suzuki_i': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'suzuki_ii': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'suzuki_iii': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
        'suzuki_iv': {'out_act': 'sigmoid', 'feature_transform': 'standardize', 'target_transform': 'normalize'},
}

dataset_best_ixs = {
    'oer_plate_4098': 39,  # 'train_r2': 0.8319234403227831, 'test_r2': 0.8465640553597371
    'oer_plate_3851': 41,  # 'train_r2': 0.7398952370543461, 'test_r2': 0.900529053900117
    'oer_plate_3860': 48,  # 'train_r2': 0.9643666817683573, 'test_r2': 0.9565719423562296 
    'oer_plate_3496': 20,  # 'train_r2': 0.8847241332118894, 'test_r2': 0.9245734657163589
    'p3ht': 26,            # 'train_r2': 0.8778774142940203, 'test_r2': 0.9673166550473478
    'agnp': 22,            # 'train_r2': 0.9894920088114398, 'test_r2': 0.9829782150809259
    'thin_film': 28,       # 'train_r2': 0.9472173771719822, 'test_r2': 0.9031088775578484
    'crossed_barrel': 37,  # 'train_r2': 0.9646820277025927, 'test_r2': 0.9409044469628027
    'autoam': 49,          # 'train_r2': 0.9970140068523364, 'test_r2': 0.9908067279613181 
    'suzuki_i': 29,        # 'train_r2': 0.9815854812719683, 'test_r2': 0.9717699198558013 
    'suzuki_ii': 34,       # 'train_r2': 0.9571388181603957, 'test_r2': 0.9481749209020203
    'suzuki_iii': 3,       # 'train_r2': 0.9942279917944439, 'test_r2': 0.9977259271817904
    'suzuki_iv': 15,       # 'train_r2': 0.9904239794801154, 'test_r2': 0.9869812427882629
}

In [5]:
def get_best_scores(res, dataset):
    scores = res[dataset]['scores']
    test_r2 = []
    sum_r2 = []
    for ix, score in enumerate(scores):
        test_r2.append(score['test_r2'])
        sum_r2.append(score['test_r2']+score['train_r2'])
    best_ix_test = np.argmax(test_r2)
    best_scores_test = scores[best_ix_test]
    best_ix_sum = np.argmax(sum_r2)
    best_scores_sum = scores[best_ix_sum]
    return best_ix_test, best_scores_test, best_ix_sum, best_scores_sum


def get_hyperparams(res, dataset_name, best_ix):
    return res[dataset_name]['params'][best_ix]

In [6]:
# for dataset in dataset_names:
#     print('DATASET : ', dataset)
#     best_ix_test, best_scores_test, best_ix_sum, best_scores_sum = get_best_scores(res, dataset)
#     print('best_ix_test : ', best_ix_test)
#     print('best_scores_test : ', best_scores_test)
#     print('best_ix_sum : ', best_ix_sum)
#     print('best_scores_sum : ', best_scores_sum)
#     print('\n\n')


In [8]:
# train the networks and make predictions on train/test set

for dataset_name in dataset_names:
    d = Dataset(kind=dataset_name)
    hyperparams = get_hyperparams(res, dataset_name, dataset_best_ixs[dataset_name])
    
    model  = BayesNeuralNet(**hyperparams, out_act=dataset_params[dataset_name]['out_act'])
    emulator = Emulator(
        dataset=dataset_name, 
        model=model,
        feature_transform=dataset_params[dataset_name]['feature_transform'],
        target_transform=dataset_params[dataset_name]['target_transform']
    )
    
    print('DATASET : ', dataset_name)
    print(emulator)
    
    scores = emulator.train()
    
    train_params = d.train_set_features.to_numpy()
    train_values = d.train_set_targets.to_numpy()
    test_params = d.test_set_features.to_numpy()
    test_values = d.test_set_targets.to_numpy()
    
    train_preds = emulator.run(train_params, num_samples=50)
    test_preds  = emulator.run(test_params, num_samples=50)
    
    
    dataset_params[dataset_name]['train_preds'] = train_preds
    dataset_params[dataset_name]['test_preds'] = test_preds
    dataset_params[dataset_name]['scores'] = scores
    dataset_params[dataset_name]['emulator'] = emulator
    
    emulator.save(f'emulator_{dataset_name}_BayesNeuralNet')
    
    
    

DATASET :  oer_plate_4098
<Emulator (Dataset(kind=oer_plate_4098), model=
--> batch_size:    10
--> es_patience:   100
--> hidden_act:    leaky_relu
--> hidden_depth:  4
--> hidden_nodes:  44
--> kind:          BayesNeuralNet
--> learning_rate: 0.00247272270870916
--> max_epochs:    100000
--> out_act:       sigmoid
--> pred_int:      100
--> reg:           0.005747336344783915
--> scope:         model)>
[INFO] >>> Training model on 80% of the dataset, testing on 20%...


/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/tensorflow_probability/python/layers/util.py:102: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)
/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/tensorflow_probability/python/layers/util.py:112: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)


[INFO]     =======================================================================
[INFO]           Epoch       Train R2     Train RMSD        Test R2      Test RMSD
[INFO]     =======================================================================
[INFO]               0         -1.347          0.233         -3.190          0.272 *
[INFO]             100          0.427          0.115          0.483          0.096 *
[INFO]             200          0.593          0.097          0.557          0.089 *
[INFO]             300          0.475          0.110          0.522          0.092
[INFO]             400          0.484          0.109          0.657          0.078 *
[INFO]             500          0.340          0.124          0.441          0.099
[INFO]             600          0.466          0.111          0.545          0.090
[INFO]             700          0.549          0.102          0.483          0.096
[INFO]             800          0.447          0.113          0.543          0.

[INFO]            8400          0.563          0.101          0.631          0.081
[INFO]            8500          0.600          0.096          0.698          0.073
[INFO]            8600          0.586          0.098          0.712          0.071
[INFO]            8700          0.553          0.102          0.648          0.079
[INFO]            8800          0.696          0.084          0.756          0.066
[INFO]            8900          0.602          0.096          0.684          0.075
[INFO]            9000          0.582          0.099          0.740          0.068
[INFO]            9100          0.502          0.108          0.704          0.072
[INFO]            9200          0.525          0.105          0.675          0.076
[INFO]            9300          0.510          0.107          0.692          0.074
[INFO]            9400          0.500          0.108          0.641          0.080
[INFO]            9500          0.623          0.094          0.768          0.064
[INF

[INFO]           17300          0.571          0.100          0.689          0.074
[INFO]           17400          0.562          0.101          0.660          0.077
[INFO]           17500          0.650          0.090          0.651          0.079
[INFO]           17600          0.671          0.087          0.787          0.061
[INFO]           17700          0.629          0.093          0.787          0.061
[INFO]           17800          0.671          0.087          0.679          0.075
[INFO]           17900          0.599          0.096          0.731          0.069
[INFO]           18000          0.652          0.090          0.747          0.067
[INFO]           18100          0.650          0.090          0.642          0.080
[INFO]           18200          0.608          0.095          0.748          0.067
[INFO]           18300          0.732          0.079          0.770          0.064
[INFO]           18400          0.668          0.088          0.749          0.067
[INF

...

[WARNING] Not all parameters are within bounds
[WARNING] Not all parameters are within bounds
[WARNING] Not all parameters are within bounds
[WARNING] Not all parameters are within bounds


DATASET :  thin_film
<Emulator (Dataset(kind=thin_film), model=
--> batch_size:    50
--> es_patience:   100
--> hidden_act:    leaky_relu
--> hidden_depth:  5
--> hidden_nodes:  60
--> kind:          BayesNeuralNet
--> learning_rate: 0.0041221056953923575
--> max_epochs:    100000
--> out_act:       relu
--> pred_int:      100
--> reg:           0.05223118992361589
--> scope:         model)>
[INFO] >>> Training model on 80% of the dataset, testing on 20%...


/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/tensorflow_probability/python/layers/util.py:102: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)
/opt/anaconda3/envs/olympus/lib/python3.7/site-packages/tensorflow_probability/python/layers/util.py:112: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  trainable=trainable)


[INFO]     =======================================================================
[INFO]           Epoch       Train R2     Train RMSD        Test R2      Test RMSD
[INFO]     =======================================================================
[INFO]               0          0.008          0.200         -0.004          0.193 *
[INFO]             100          0.633          0.121          0.805          0.085 *
[INFO]             200          0.689          0.112          0.831          0.079 *
[INFO]             300          0.679          0.114          0.815          0.083
[INFO]             400          0.670          0.115          0.757          0.095
[INFO]             500          0.706          0.109          0.836          0.078 *
[INFO]             600          0.760          0.098          0.764          0.094
[INFO]             700          0.768          0.096          0.829          0.080
[INFO]             800          0.803          0.089          0.834          0.

KeyboardInterrupt: 

In [ ]:
_olympus_reference_colors = [
    "#08294C",
    "#75BBE1",
    "#D4E9F4",
    "#F2F2F2",
    "#F7A4D4",
    "#F75BB6",
    "#EB0789",
]

dataset_targets = {
    'oer_plate_4098': 'overpotential [V]',  
    'oer_plate_3851': 'overpotential [V]',  
    'oer_plate_3860': 'overpotential [V]',  
    'oer_plate_3496': 'overpotential [V]',  
    'p3ht': 'conductivity',           
    'agnp': 'spectrum score',           
    'thin_film': 'instability index',       
    'crossed_barrel': 'mechanical toughness',  
    'autoam': 'shape score',           
    'suzuki_i': ['yield [%]', '[prod/cat]'],        
    'suzuki_ii': ['yield [%]', '[prod/cat]'],       
    'suzuki_iii': ['yield [%]', '[prod/cat]'],       
    'suzuki_iv': ['yield [%]', '[prod/cat]'],       
}

In [ ]:
# make plots for single objective datasets

fig, axes = plt.subplots(2, 5, figsize=(18, 6))
axes = axes.flatten()

for ix, dataset_name in enumerate(dataset_names):
    if 'suzuki_' not in dataset_name:
        d = Dataset(kind=dataset_name)
        print(d.data)
        emualtor = Emulator(dataset=dataset_name, model='BayesNeuralNet')
        print(emulator)
        train_params = d.train_set_features.to_numpy()
        train_values = d.train_set_targets.to_numpy()
        test_params = d.test_set_features.to_numpy()
        test_values = d.test_set_targets.to_numpy()

        train_preds = emulator.run(train_params, num_samples=50)
        test_preds  = emulator.run(test_params, num_samples=50)
        
        meas_name = d.target_names
        
        # training data 
        axes[ix].plot(
            train_values,
            train_preds,
            c="#75BBE1",
            ls='',
            marker='o',
            markersize=4,
        )
        
        # training data 
        axes[ix].plot(
            test_values,
            test_preds,
            c="#EB0789",
            ls='',
            marker='o',
            markersize=4,
        )  
        
        axes[ix].set_xlabel(f'true {dataset_targets[dataset_name]}')
        axes[ix].set_ylabel(f'pred {dataset_targets[dataset_name]}')
    else:
        pass
        
        
plt.tight_layout()
        

In [ ]:
dataset_params['p3ht']['scores']

In [ ]:
emualtor = Emulator(dataset='p3ht', model='BayesNeuralNet')
print(emulator)

In [9]:
d = Dataset(kind='p3ht')
ix0 = d.test_indices

In [10]:
d = Dataset(kind='p3ht')
ix1 = d.test_indices

In [11]:
ix0

array([158, 134,  32, 122,  74,  75, 119,  28,  17,  68, 163, 137, 160,
        62, 118,  45,  79,   6,  82, 133,   2, 177, 107, 173, 138,  31,
       168,  55, 171, 125, 110,  47, 142, 115, 147,  66])

In [12]:
ix1

array([ 88,  12, 162, 148,  77,  51,   3,  36,  86, 135, 130,  81,  11,
         6, 176, 103,  61, 154,  14,  62,  91,  30,  70,  29,  87, 150,
       106,  26,  72,  35,  44, 119,  19, 132,  37, 174])